In [26]:
import pandas as pd
import re
from datetime import datetime, timedelta
import os


In [27]:
os.getcwd()


'/content'

In [28]:
os.listdir('/content')


['.config',
 'customer_support_tickets.csv',
 'final_ticket_triage_output.csv',
 'support_manager_report.csv',
 'sample_data']

In [29]:
df = pd.read_csv('/content/customer_support_tickets.csv')
df.head()


,Ticket ID,Customer Name,Customer Email,Customer Age,Customer Gender,Product Purchased,Date of Purchase,Ticket Type,Ticket Subject,Ticket Description,Ticket Status,Resolution,Ticket Priority,Ticket Channel,First Response Time,Time to Resolution,Customer Satisfaction Rating
0,1,Marisa Obrien,carrollallison@example.com,32,Other,GoPro Hero,2021-03-22,Technical issue,Product setup,I'm having an issue with the {product_purchase...,Pending Customer Response,NaN,Critical,Social media,2023-06-01 12:15:36,NaN,NaN
1,2,Jessica Rios,clarkeashley@example.com,42,Female,LG Smart TV,2021-05-22,Technical issue,Peripheral compatibility,I'm having an issue with the {product_purchase...,Pending Customer Response,NaN,Critical,Chat,2023-06-01 16:45:38,NaN,NaN
2,3,Christopher Robbins,gonzalestracy@example.com,48,Other,Dell XPS,2020-07-14,Technical issue,Network problem,I'm facing a problem with my {product_purchase...,Closed,Case maybe show recently my computer follow.,Low,Social media,2023-06-01 11:14:38,2023-06-01 18:05:38,3.0
3,4,Christina Dillon,bradleyolson@example.org,27,Female,Microsoft Office,2020-11-13,Billing inquiry,Account access,I'm having an issue with the {product_purchase...,Closed,Try capital clearly never color toward story.,Low,Social media,2023-06-01 07:29:40,2023-06-01 01:57:40,3.0
4,5,Alexander Carroll,bradleymark@example.com,67,Female,Autodesk AutoCAD,2020-02-04,Billing inquiry,Data loss,I'm having an issue with the {product_purchase...,Closed,West decision evidence bit.,Low,Email,2023-06-01 00:12:42,2023-06-01 19:53:42,1.0


In [30]:
df.info()
df.isnull().sum()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8469 entries, 0 to 8468
Data columns (total 17 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Ticket ID                     8469 non-null   int64  
 1   Customer Name                 8469 non-null   object 
 2   Customer Email                8469 non-null   object 
 3   Customer Age                  8469 non-null   int64  
 4   Customer Gender               8469 non-null   object 
 5   Product Purchased             8469 non-null   object 
 6   Date of Purchase              8469 non-null   object 
 7   Ticket Type                   8469 non-null   object 
 8   Ticket Subject                8469 non-null   object 
 9   Ticket Description            8469 non-null   object 
 10  Ticket Status                 8469 non-null   object 
 11  Resolution                    2769 non-null   object 
 12  Ticket Priority               8469 non-null   object 
 13  Tic

,0
Ticket ID,0
Customer Name,0
Customer Email,0
Customer Age,0
Customer Gender,0
Product Purchased,0
Date of Purchase,0
Ticket Type,0
Ticket Subject,0
Ticket Description,0


In [31]:
df.columns = df.columns.str.lower().str.replace(' ', '_')
df.columns


Index(['ticket_id', 'customer_name', 'customer_email', 'customer_age',
       'customer_gender', 'product_purchased', 'date_of_purchase',
       'ticket_type', 'ticket_subject', 'ticket_description', 'ticket_status',
       'resolution', 'ticket_priority', 'ticket_channel',
       'first_response_time', 'time_to_resolution',
       'customer_satisfaction_rating'],
      dtype='object')

In [32]:
def clean_text(text):
    text = str(text).lower()
    text = re.sub(r'[^a-z\s]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text


In [33]:
df['clean_message'] = df['ticket_description'].apply(clean_text)

df[['ticket_description', 'clean_message']].head()


,ticket_description,clean_message
0,I'm having an issue with the {product_purchase...,im having an issue with the productpurchased p...
1,I'm having an issue with the {product_purchase...,im having an issue with the productpurchased p...
2,I'm facing a problem with my {product_purchase...,im facing a problem with my productpurchased t...
3,I'm having an issue with the {product_purchase...,im having an issue with the productpurchased p...
4,I'm having an issue with the {product_purchase...,im having an issue with the productpurchased p...


In [34]:
def classify_issue(text):
    if any(word in text for word in ['payment', 'billing', 'charge', 'card']):
        return 'PAYMENT'
    elif any(word in text for word in ['login', 'password', 'account', 'access']):
        return 'LOGIN'
    elif any(word in text for word in ['delivery', 'shipment', 'delay']):
        return 'DELIVERY'
    elif any(word in text for word in ['refund', 'return']):
        return 'REFUND'
    elif any(word in text for word in ['error', 'bug', 'issue', 'problem', 'crash']):
        return 'BUG'
    else:
        return 'GENERAL'

df['issue_type'] = df['clean_message'].apply(classify_issue)


In [35]:
def assign_priority(text):
    if any(word in text for word in ['urgent', 'critical', 'immediately', 'asap']):
        return 'P0'
    elif any(word in text for word in ['not working', 'error', 'failed']):
        return 'P1'
    elif any(word in text for word in ['delay', 'slow']):
        return 'P2'
    else:
        return 'P3'

df['assigned_priority'] = df['clean_message'].apply(assign_priority)


In [36]:
sla_map = {
    'P0': 2,
    'P1': 6,
    'P2': 24,
    'P3': 48
}

df['sla_hours'] = df['assigned_priority'].map(sla_map)


In [37]:
df['created_time'] = datetime.now()
df['due_time'] = df['created_time'] + df['sla_hours'].apply(lambda x: timedelta(hours=x))


In [38]:
support_report = (
    df.groupby(['issue_type', 'assigned_priority'])
      .size()
      .reset_index(name='ticket_count')
)

support_report


,issue_type,assigned_priority,ticket_count
0,BUG,P0,147
1,BUG,P1,712
2,BUG,P2,1
3,BUG,P3,6018
4,DELIVERY,P0,1
5,DELIVERY,P1,4
6,DELIVERY,P2,4
7,DELIVERY,P3,27
8,GENERAL,P0,31
9,GENERAL,P3,39


In [39]:
df[['ticket_id', 'issue_type', 'assigned_priority', 'sla_hours', 'due_time']].head()


,ticket_id,issue_type,assigned_priority,sla_hours,due_time
0,1,PAYMENT,P3,48,2025-12-29 15:49:41.834214
1,2,BUG,P3,48,2025-12-29 15:49:41.834214
2,3,PAYMENT,P3,48,2025-12-29 15:49:41.834214
3,4,BUG,P3,48,2025-12-29 15:49:41.834214
4,5,DELIVERY,P3,48,2025-12-29 15:49:41.834214


In [40]:
df.to_csv('final_ticket_triage_output.csv', index=False)
support_report.to_csv('support_manager_report.csv', index=False)
